In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [62]:
input_text = au.read_txt_file_lines('input.txt')
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

for ic, col in enumerate(input_text[0]):
    if ic != 1:
        assert col == '#', ic
for ic, col in enumerate(input_text[-1]):
    if ic != n_cols - 2:
        assert col == '#', ic
for row in input_text:
    assert row[0] == '#', f'row {row} not valid'
    assert row[-1] == '#', f'row {row} not valid'

unique_chars = set()
for row in input_text:
    unique_chars.update(row)
print(f'unique_chars: {unique_chars}')

input has 141 rows and 141 cols
unique_chars: {'v', '.', '#', '>'}


## Part 1:
- `path_tuple` keeps track of a path: the current head (x, y) and previously visited nodes (set of nodes)
- `progress_til_junction` moves path from its start position (`head`) to next junction. Not allowing previously visited nodes and sliding down `>` and `v`. Returns True/False whether end_node is reached, and path. At a junction, it returns a list of new paths. 
- `find_longest_path`: uses a deque to go through all possible paths. Stores paths that reach end node. Then find answers by picking path with longest length. 

In [15]:
path_tuple = collections.namedtuple('path', 'head visited_nodes')

def progress_til_junction(path, input_text, end_node=(n_rows - 1, n_cols - 2)):
    n_rows, n_cols = len(input_text), len(input_text[0])
    current_row, current_col = path.head
    while True:
        possible_next_nodes = []
        if input_text[current_row][current_col] == '>':
            if current_col < n_cols - 1 and input_text[current_row][current_col + 1] in ['.', '>']:
                possible_next_nodes.append((current_row, current_col + 1))
        elif input_text[current_row][current_col] == 'v':
            if current_row < n_rows - 1 and input_text[current_row + 1][current_col] in ['.', 'v']:
                possible_next_nodes.append((current_row + 1, current_col))
        else:
            if current_col < n_cols - 1 and input_text[current_row][current_col + 1] in ['.', '>']:
                possible_next_nodes.append((current_row, current_col + 1))
            if current_col > 0 and input_text[current_row][current_col - 1] == '.':
                possible_next_nodes.append((current_row, current_col - 1))
            if current_row < n_rows - 1 and input_text[current_row + 1][current_col] in ['.', 'v']:
                possible_next_nodes.append((current_row + 1, current_col))
            if current_row > 0 and input_text[current_row - 1][current_col] == '.':
                possible_next_nodes.append((current_row - 1, current_col))

        possible_next_nodes = [node for node in possible_next_nodes if node not in path.visited_nodes]

        if len(possible_next_nodes) == 0:
            if path.head == end_node:
                return True, path
            else:
                print(f'path {path} has no possible next nodes')
                return False, []
        elif len(possible_next_nodes) == 1:
            new_visited_nodes = copy.deepcopy(path.visited_nodes)
            new_visited_nodes.add(path.head)
            path = path._replace(head=possible_next_nodes[0])
            path = path._replace(visited_nodes=new_visited_nodes)
            current_row, current_col = path.head
        else:
            new_paths = []
            for node in possible_next_nodes:
                new_visited_nodes = copy.deepcopy(path.visited_nodes)
                new_visited_nodes.add(path.head)
                new_paths.append(path_tuple(node, new_visited_nodes))
            return False, new_paths 
        
def find_longest_path(input_text, start_row=0, start_col=1, verbose=1, part2=False):
    visited_nodes = set()
    visited_nodes.add((start_row, start_col))
    queue_paths = collections.deque()
    queue_paths.append(path_tuple((start_row, start_col), visited_nodes))
    paths_to_end = []
    i_path = 0
    while len(queue_paths) > 0: 
        i_path += 1
        current_path = queue_paths.popleft()
        if verbose > 0:
            print(f'starting new path (#{i_path}), starting at {current_path.head}. {len(queue_paths)} paths left in queue')
        if part2: 
            reached_end, new_path = progress_til_junction_part2(current_path, input_text)
        else:
            reached_end, new_path = progress_til_junction(current_path, input_text)
        if reached_end:
            if  verbose > 1:
                print(f'found end {new_path}')
            paths_to_end.append(new_path)
        else:
            if verbose > 1:
                print(f'adding {len(new_path)} new paths')
            queue_paths.extend(new_path)

    if verbose > 0:
        print(f'found {len(paths_to_end)} paths to end')
    return paths_to_end

In [5]:
paths_to_end = find_longest_path(input_text, verbose=1)
paths_length = [len(path.visited_nodes) for path in paths_to_end]
max_path_length = max(paths_length)
print(f'max_path_length: {max_path_length}')

starting new path (#1), starting at (0, 1). 0 paths left in queue
starting new path (#2), starting at (5, 4). 1 paths left in queue
starting new path (#3), starting at (6, 3). 2 paths left in queue
starting new path (#4), starting at (3, 12). 3 paths left in queue
starting new path (#5), starting at (4, 11). 2 paths left in queue
starting new path (#6), starting at (13, 6). 3 paths left in queue
starting new path (#7), starting at (14, 5). 4 paths left in queue
starting new path (#8), starting at (13, 14). 3 paths left in queue
starting new path (#9), starting at (14, 13). 2 paths left in queue
starting new path (#10), starting at (13, 14). 1 paths left in queue
starting new path (#11), starting at (14, 13). 0 paths left in queue
found 6 paths to end
max_path_length: 94


## part 2:
Bit tricky. Brute-forcing the previous solution did not work as expected. Instead, create weighted graph of junctions. This should save a lot of time going through the maze. 
- `find_section_between_junctions` creates an edge. Start & end node, and distance are returned. 
- `map_all_edges` goes through entire maze and creates a list of all edges. 
Then, create a distance matrix (not sure if best format?) Then go use algorithm of part 1 (Dijkstra) to go through weighted graph. Works great for sample input, but for test input it's too slow... Tried the following tricks:
1. Every 100(?) iterations, re-sort the deque by distance (per path). Seems to be a pretty good heuristic for prioritising finishing/finding the longest paths. Worked well, quickly found local max (6470) but didn improve even after running for minutes. However this was not the answer apparently. 
2. Reverse search. Start at end point and end at start. Quickly found one better path: 6534. And that was the answer!! 

In [63]:
input_text2 = []
for row in input_text:
    input_text2.append(row.replace('>', '.').replace('v', '.'))

assert len(input_text2) == len(input_text)
assert len(input_text2[0]) == len(input_text[0])

unique_chars = set()
for row in input_text2:
    unique_chars.update(row)
print(f'unique_chars: {unique_chars}')

unique_chars: {'.', '#'}


In [64]:
class Path2:
    def __init__(self, head, visited_nodes):
        assert type(visited_nodes) == list and len(visited_nodes) == 1
        self.head = head 
        self.visited_nodes = visited_nodes

network_edge = collections.namedtuple('network_edge', 'node_1 node_2 dist')
        
def find_section_between_junctions(input_text, path, verbose=0):
    n_rows, n_cols = len(input_text), len(input_text[0])
    current_row, current_col = path.head
    end_node = (n_rows - 1, n_cols - 2)
    assert len(path.visited_nodes) == 1, path
    while True:
        if verbose > 0:
            print(f'new it. head {path.head}; nodes visited: {path.visited_nodes}')
        possible_next_nodes = []
    
        if current_col < n_cols - 1 and input_text[current_row][current_col + 1] == '.':
            possible_next_nodes.append((current_row, current_col + 1))
        if current_col > 0 and input_text[current_row][current_col - 1] == '.':
            possible_next_nodes.append((current_row, current_col - 1))
        if current_row < n_rows - 1 and input_text[current_row + 1][current_col] == '.':
            possible_next_nodes.append((current_row + 1, current_col))
        if current_row > 0 and input_text[current_row - 1][current_col] == '.':
            possible_next_nodes.append((current_row - 1, current_col))

        possible_next_nodes = [node for node in possible_next_nodes if node not in path.visited_nodes]
        path.visited_nodes.append(path.head)
            
        if len(possible_next_nodes) == 0:
            if path.head == end_node:
                start_node = path.visited_nodes[0]
                n_nodes = len(path.visited_nodes) - 1
                new_edge = network_edge(start_node, end_node, n_nodes)
                # print(new_edge)
                return True, [], set([new_edge])
            else:
                if verbose > 0:
                    print(f'path {path} has no possible next nodes')
                return False, [], set()
        elif len(possible_next_nodes) == 1:
            path.head = possible_next_nodes[0]
            current_row, current_col = path.head
        else:
            if verbose > 0:
                print(f'reached junction. head: {path.head}. next: {possible_next_nodes}')
            new_edges = []
            new_paths = []
            for i_node, node in enumerate(possible_next_nodes):
                start_node_path = path.visited_nodes[0]
                end_node_path = path.visited_nodes[-1]
                n_nodes = len(path.visited_nodes) - 1
                new_single_path = Path2(node, [path.head])
                # if i_node == 0:
                new_edge = network_edge(start_node_path, end_node_path, n_nodes)
                new_edges.append(new_edge)
                new_paths.append(new_single_path)
            return False, new_paths, set(new_edges)
        
def map_all_edges(input_text, start_row=0, start_col=1, verbose=1):
    visited_nodes = set()
    visited_nodes.add((start_row, start_col))
    queue_paths = collections.deque()
    start_path = Path2((start_row + 1, start_col), [(start_row, start_col)])
    queue_paths.append(start_path)
    set_edges = set()
    i_path = 0 
    while len(queue_paths) > 0:
        i_path += 1
        current_path = queue_paths.popleft()
        if verbose > 0:
            print(f'starting new path (#{i_path}), starting at {current_path.head}. {len(queue_paths)} paths left in queue')
        reached_end, new_paths, new_edges = find_section_between_junctions(input_text=input_text, 
                                                                           path=current_path)     
        set_edges.update(new_edges)

        set_junction_nodes = set()
        for p in new_paths:
            junction_node = p.visited_nodes[0]
            if junction_node not in visited_nodes:
                queue_paths.append(p)
                set_junction_nodes.add(junction_node)

        visited_nodes.update(set_junction_nodes)

        if reached_end:
            expected_end_node = (n_rows - 1, n_cols -2)
            assert len(new_edges) == 1
            assert list(new_edges)[0].node_2 == expected_end_node
            visited_nodes.add(expected_end_node)
              
    return set_edges, visited_nodes



In [65]:
set_edges, set_nodes = map_all_edges(input_text=input_text2, verbose=0)
n_edges = len(set_edges)
n_nodes = len(set_nodes)
n_rows = len(input_text)
n_cols = len(input_text[0])
assert (0, 1) in set_nodes and (n_rows - 1, n_cols - 2) in set_nodes, set_nodes
print(f'{n_edges} edges and {n_nodes} nodes found')

85 edges and 36 nodes found


In [66]:
dist_mat = np.zeros((n_nodes, n_nodes))
dist_mat += np.nan 
dict_edges = {}
list_nodes = list(set_nodes)
start_node = (0, 1)
end_node = (n_rows - 1, n_cols - 2)
list_nodes.remove(start_node)
list_nodes.remove(end_node)
list_nodes = [start_node] + list_nodes + [end_node]
for i_edge, edge in enumerate(set_edges):
    assert edge.node_1 in list_nodes and edge.node_2 in list_nodes 
    ind_node_1 = list_nodes.index(edge.node_1)
    ind_node_2 = list_nodes.index(edge.node_2)
    dist_mat[ind_node_1, ind_node_2] = edge.dist
    dist_mat[ind_node_2, ind_node_1] = edge.dist
print(list_nodes)
dist_mat

[(0, 1), (17, 19), (35, 131), (41, 79), (131, 103), (53, 129), (43, 41), (11, 61), (9, 29), (77, 17), (99, 31), (85, 137), (123, 43), (131, 83), (61, 37), (41, 63), (53, 17), (89, 55), (13, 85), (53, 109), (107, 13), (77, 85), (35, 99), (75, 107), (65, 55), (63, 83), (37, 13), (101, 77), (131, 53), (113, 67), (111, 135), (107, 111), (123, 135), (87, 39), (7, 105), (140, 139)]


array([[ nan, 151.,  nan, ...,  nan,  nan,  nan],
       [151.,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan, 434.,  nan],
       ...,
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan, 434., ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [87]:
class PathGraph:
    def __init__(self, head, visited_nodes):
        self.head = head 
        self.visited_nodes = visited_nodes

    def __repr__(self) -> str:
        return ' '.join([str(x) for x in self.visited_nodes] + [str(self.head)])

def progress_til_junction_nodes(path, dist_mat, list_nodes, end_node=(n_rows - 1, n_cols - 2),
                                verbose=1):
    current_node = path.head
    while True:
        ind_current_node = list_nodes.index(current_node)
        possible_next_nodes = np.where(~np.isnan(dist_mat[ind_current_node, :]))[0]
        possible_next_nodes = [list_nodes[x] for x in possible_next_nodes if list_nodes[x] not in path.visited_nodes]
        if verbose > 0:
            print(f'from {current_node}, there are {len(possible_next_nodes)} follow ups ({possible_next_nodes})')
        if len(possible_next_nodes) == 0:
            if path.head == end_node:
                path.visited_nodes.append(end_node)
                if verbose > 0:
                    print('reached end')
                return True, path
            else:
                if verbose > 0:
                    print(f'path {path} has no possible next nodes')
                return False, []
        elif len(possible_next_nodes) == 1:
            path.visited_nodes.append(path.head)
            path.head = possible_next_nodes[0]
            current_node = path.head
            if verbose > 0: 
                print(f'only 1 possibility, moving on to {current_node}')
        else:
            if verbose > 0:
                print('splitting')
            new_paths = []
            # list_upcoming_dists = []
            for node in possible_next_nodes:
                new_visited_nodes = copy.deepcopy(path.visited_nodes)
                new_visited_nodes.append(path.head)
                # ind_new_node = list_nodes.index(path.head)
                # upcoming_dist = dist_mat[ind_current_node, ind_new_node]
                # list_upcoming_dists.append(upcoming_dist)
                new_paths.append(PathGraph(node, new_visited_nodes))
            return False, new_paths 

def find_longest_path_part2(list_nodes, dist_mat, verbose=0, max_it=1000000, reverse_search=False):
    queue_paths = collections.deque()
    if reverse_search:
        queue_paths.append(PathGraph(list_nodes[-1], list()))
        end_node = (0, 1)
    else:
        queue_paths.append(PathGraph(list_nodes[0], list()))
        end_node = (n_rows - 1, n_cols - 2)
    paths_to_end = []
    i_path = 0
    max_dist_so_far = 0
    while len(queue_paths) > 0: 
        i_path += 1
        if i_path == max_it:
            break
        # if i_path == 5:
        #     return queue_paths

        current_path = queue_paths.popleft()
        if verbose > 0 and i_path % 10000 == 0:
            print(f'starting new path (#{i_path}), starting at {current_path.head}. {len(queue_paths)} paths left in queue')
        
        if i_path > 0 and i_path % 100 == 0:
            queue_paths = sorted(queue_paths, key=lambda x: get_total_dist_per_edge(x, list_nodes, dist_mat), reverse=True)
            queue_paths = collections.deque(queue_paths)

        reached_end, new_path = progress_til_junction_nodes(current_path, dist_mat, list_nodes, verbose=0,
                                                            end_node=end_node)
        
        if reached_end:
            if verbose > 1:
                print(f'found end {new_path}')
            paths_to_end.append(new_path)
            final_dist = get_total_dist_per_edge(new_path, list_nodes=list_nodes, dist_mat=dist_mat)
            if final_dist > max_dist_so_far:
                max_dist_so_far = final_dist 
                print(f'NEW BEST: {max_dist_so_far}')
        else:
            if verbose > 1:
                print(f'adding {len(new_path)} new paths')
            for p in new_path:
                if p not in queue_paths:
                    queue_paths.append(p)

    if verbose > 0:
        print(f'found {len(paths_to_end)} paths to end')
    return paths_to_end

def get_dist_for_all_edges(list_paths, list_nodes, dist_mat):
    list_dist = [] 
    for p in list_paths:
        list_dist.append(get_total_dist_per_edge(p, list_nodes=list_nodes, dist_mat=dist_mat))
    return list_dist 

def get_total_dist_per_edge(p, list_nodes, dist_mat):
    dist = 0 
    for i_node in range(len(p.visited_nodes) - 1):
        node_1 = p.visited_nodes[i_node]
        node_2 = p.visited_nodes[i_node + 1]
        ind_1 = list_nodes.index(node_1)
        ind_2 = list_nodes.index(node_2)
        dist += dist_mat[ind_1, ind_2]
    return dist 

In [ ]:
list_paths = find_longest_path_part2(list_nodes=list_nodes, dist_mat=dist_mat, verbose=1,
                                     max_it=1000000, reverse_search=True)

In [91]:
list_dists = get_dist_for_all_edges(list_paths, list_nodes, dist_mat)
max(list_dists)

6534.0